In [ ]:
%pip install -r requirements.txt

In [ ]:
!git clone https://github.com/qwopqwop200/GPTQ-for-LLaMa gptq

In [ ]:
%tb

In [ ]:
from guider_server import run
# run('TheBloke/tulu-13B-GPTQ', 'gptq_model-4bit-128g')
run('TheBloke/Wizard-Vicuna-13B-Uncensored-GPTQ', 'Wizard-Vicuna-13B-Uncensored-GPTQ-4bit-128g.compat.no-act-order')